In [1]:
import sys
sys.path.append("C:\Program Files\Tinysoft\Analyse.NET")
sys.path.append(r"C:\Users\xudong\Documents\github\coresearch\funcs")
import pymongo
import numpy as np
import pickle
import time
import datetime
import copy
import pandas as pd
from operator import itemgetter
import TSLPy3 as tsl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import talib
import importlib
import rschLib
np.set_printoptions(formatter={'float_kind': "{:.6f}".format})
client = pymongo.MongoClient('localhost', 27017)
db = rschLib.db_quanLiang()
dbt = rschLib.db_tinySoftData()
dtes, tkrs, name, open_mtx, high_mtx, low_mtx, close_mtx,belong, shenwan1, shenwan2, shenwan3, vol_mtx, amount_mtx = rschLib.loadDailyBarMtx()
# get time labels
# get trades



In [2]:
q = db.taskAddTagToStrategy.find_one({'lastUpdate':{'$exists':False}, 'status':'new'})
if q==None:
    q = db.taskAddTagToStrategy.find_one({'lastUpdate':{'$lt':str(datetime.datetime.now().year)+str(datetime.datetime.now().month)+str(datetime.datetime.now().day)}})
    if q==None:
        return
strategy_name_original = q['strategy_name_original']
strategy_name = q['strategy_name']
inTime = timeLabels.index(q['selectInTime'])
otTime = timeLabels.index(q['selectOtTime'])
tradeArea=[inTime,otTime]

{'_id': ObjectId('5fd72095161c07e1ec61e4df'),
 'strategy_name': '第2天09:31,第3天13:01,tagNetProfitGrowth3YBiggerThan20P,tagCirculateMarketValueBiggerThan500Y',
 'userName': '董旭',
 'selectInTime': '第2天09:31',
 'selectOtTime': '第3天13:01',
 'status': 'new',
 'strategy_name_original': 'tagNetProfitGrowth3YBiggerThan20P',
 'tagToAdd': ['tagCirculateMarketValueBiggerThan500Y']}

In [ ]:
while (1==1):
    q = db.taskAddTagToStrategy.find_one({'lastUpdate':{'$exists':False}, 'status':'new'})
    if q==None:
        #q = db.taskAddTagToStrategy.find_one({'lastUpdate':{'$lt':str(datetime.datetime.now().year)+str(datetime.datetime.now().month)+str(datetime.datetime.now().day)}})
        #if q==None:
        time.sleep(5)
        continue
    strategy_name_original = q['strategy_name_original']
    strategy_name = q['strategy_name']
    inTime = timeLabels.index(q['selectInTime'])
    otTime = timeLabels.index(q['selectOtTime'])
    tradeArea=[inTime,otTime]
    maxD = 5
    timeAsFloat, timeLabels, maxM, dayOff, dayTimeAsFloat = rschLib.getTimeLabels(maxD)
    R = open_mtx[:, 1:]/close_mtx[:,:-1]-1  #使用收盘到开盘的回报率来修正分红和拆股
    R = np.hstack((np.zeros((R.shape[0],1)), R))
    importlib.reload(rschLib)
    tradesUsedOriginal, r_withnanOriginal = rschLib.getTradesFast(strategy_name_original, name, tkrs, dtes, maxD, dayTimeAsFloat, R)
    m = db.strategyBackTest.find_one({'strategy_name':strategy_name_original})
    if ('off_start' in m.keys()):
        offStart = (m['off_start'][0], m['off_start'][1])
    d = rschLib.tagDict()
    idxTradesOverLappedList = []
    for i in range(len(q['tagToAdd'])):
        k = list(d.keys())
        t = list([x['file'] for x in d.values()])
        t.index(q['tagToAdd'][i])
        tagName = k[t.index(q['tagToAdd'][i])]
        tag_off_start = d[tagName]['off_start']
        with open("D:\\pklWeeklyUpdate\\" + q['tagToAdd'][i] + ".pkl", 'rb+') as f:
            tagFile = pickle.load(f)
            tagMtx = tagFile['tag_mtx']    
        [p,y] = rschLib.totInTag(range(len(tradesUsedOriginal)), tagMtx, dtes, tkrs, tradesUsedOriginal, offStart, tag_off_start)
        idxTradesOverLappedList.append(y)
    idxTradesOverLapped = idxTradesOverLappedList[0]
    for x in idxTradesOverLappedList:
        idxTradesOverLapped = list(set(idxTradesOverLapped).intersection(x))
        idxTradesOverLapped.sort()
    tradesUsed = [tradesUsedOriginal[x] for x in idxTradesOverLapped]
    r_withnan = r_withnanOriginal[idxTradesOverLapped, :]
    r = r_withnan.copy()
    r[np.isfinite(r)==False]=0
    # draw price change
    idxTradable = np.isfinite(r_withnan[:,tradeArea[0]])
    result = rschLib.getTradeAnalysisSampleGroups(r, idxTradable, tradeArea)
    rschLib.drawPriceChange(r[idxTradable,:], strategy_name, timeLabels=timeLabels, tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rGood10'], strategy_name, timeLabels=timeLabels, title='盈利前10%交易', tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rGood30'], strategy_name, timeLabels=timeLabels, title='盈利前30%交易', tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rBad10'], strategy_name, timeLabels=timeLabels, title='亏损前10%交易',  tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rBad30'], strategy_name, timeLabels=timeLabels, title='亏损前30%交易',  tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rDieting'], strategy_name, timeLabels=timeLabels, title='第一天跌停', tp=tradeArea, showGraph='no')
    # analyze tags
    rschLib.analyzeTradeTags(tradesUsed, result['rGood10'], result['idxGood10'], '盈利前10%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    rschLib.analyzeTradeTags(tradesUsed, result['rGood30'], result['idxGood30'], '盈利前30%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    rschLib.analyzeTradeTags(tradesUsed, result['rBad10'], result['idxBad10'], '亏损前10%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    rschLib.analyzeTradeTags(tradesUsed, result['rBad30'], result['idxBad30'], '亏损前30%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    rschLib.analyzeTradeTags(tradesUsed, result['rDieting'], result['idxDieting'], '第一天跌停',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #get tag names
    tnames, tagNamesEn,t2 = rschLib.getTagNames()
    idxOverLapTagList=rschLib.analyzeTradeTags(tradesUsed, r, list(range(len(tradesUsed))), '所有交易',strategy_name, dtes, tkrs, offStart, "d:\\pklWeeklyUpdate\\")
    #draw pnl and tag pnl
    importlib.reload(rschLib)
    [dtesByTrade, pnlByTrade] = rschLib.getPnlFast(r, idxTradable, dtes, tkrs, name, tradesUsed, inTime, otTime, dayOff, timeAsFloat, toDatabase='yes', strategy_name=strategy_name)
    [dtesPnlAggr,pnlAggr, numTrades] = rschLib.aggregatePnlAndDtes(dtesByTrade, pnlByTrade)
    rschLib.drawPNL(dtesPnlAggr, pnlAggr, dtes, strategy_name,showFigure='no', toDatabase='yes')
    for i in range(len(tnames)):
        tagName = tnames[i]
        [dtesWithTag, pnlWithTag,n] = rschLib.aggregatePnlAndDtes(dtesByTrade[idxOverLapTagList[i]],pnlByTrade[idxOverLapTagList[i]])
        rschLib.drawPNL(dtesWithTag, pnlWithTag, dtes, strategy_name, showFigure='no', toDatabase='yes', dateStart=dtesPnlAggr[0], pnlType=tagName)
        rschLib.drawPNL(dtesWithTag, pnlWithTag, dtes, strategy_name+'+'+tagNamesEn[i], showFigure='no',  toDatabase='yes', dateStart=dtesPnlAggr[0], pnlType='pnl')
        # 求非涨停，以及伴随有tagName标签的交易，绘制价格变化曲线
        idxTradableAndHasTag = list(np.intersect1d(np.nonzero(idxTradable)[0], np.array(idxOverLapTagList[i])))
        rschLib.drawPriceChange(r[idxTradableAndHasTag,:], strategy_name, timeLabels=timeLabels, title=tagName,  tp=tradeArea, showGraph='no')
    # control group and optimal group
    [rawInTime, rawOtTime] = rschLib.getDefaultTradeTime(offStart)
    idxTradableRaw = np.isfinite(r_withnan[:, rawInTime])
    [dtesByTradeRaw, pnlByTradeRaw] = rschLib.getPnlFast(r, idxTradableRaw, dtes, tkrs, name, tradesUsed, rawInTime, rawOtTime, dayOff, timeAsFloat, toDatabase='yes', strategy_name=strategy_name)
    [dtesPnlAggrRaw, pnlAggrRaw, numTradesRaw] = rschLib.aggregatePnlAndDtes(dtesByTradeRaw, pnlByTradeRaw)
    rschLib.drawPNL(dtesPnlAggrRaw, pnlAggrRaw, dtes, strategy_name, showFigure='yes', toDatabase='yes', pnlType='rawPnl')
    [bestInTime, bestOtTime] = rschLib.getOptimalTradeTime(r[idxTradable, :], rawInTime)
    idxTradableBestInTime = np.isfinite(r_withnan[:, bestInTime])
    [dtesByTradeBest, pnlByTradeBest] = rschLib.getPnlFast(r, idxTradableBestInTime, dtes, tkrs, name, tradesUsed, bestInTime, bestOtTime, dayOff, timeAsFloat, toDatabase='no', strategy_name=strategy_name)
    [dtesPnlAggrBest, pnlAggrBest, k] = rschLib.aggregatePnlAndDtes(dtesByTradeBest, pnlByTradeBest)
    rschLib.drawPNL(dtesPnlAggrBest, pnlAggrBest, dtes, strategy_name, showFigure='yes', toDatabase='yes', pnlType='optimalTradeTimePnl')
    rschLib.updateStrategyBackTest(strategy_name, 'bestInTimePoint', int(bestInTime))
    rschLib.updateStrategyBackTest(strategy_name, 'bestOtTimePoint', int(bestOtTime))
    rschLib.updateStrategyBackTest(strategy_name, 'bestInTimeLabel', timeLabels[bestInTime])
    rschLib.updateStrategyBackTest(strategy_name, 'bestOtTimeLabel', timeLabels[bestOtTime])
    rschLib.updateStrategyBackTest(strategy_name, 'rawInTimePoint', int(rawInTime))
    rschLib.updateStrategyBackTest(strategy_name, 'rawOtTimePoint', int(rawOtTime))
    rschLib.updateStrategyBackTest(strategy_name, 'rawInTimeLabel', timeLabels[rawInTime])
    rschLib.updateStrategyBackTest(strategy_name, 'rawOtTimeLabel', timeLabels[rawOtTime])
    #analysis of number of trades vs performance
    importlib.reload(rschLib)
    rschLib.pnlVsNumtrades(pnlAggr, numTrades, strategy_name, toDatabase='yes')
    rschLib.saveOffStart(strategy_name, offStart)
    db.taskAddTagToStrategy.update_one({'_id':q['_id']}, {'$set':{
        'lastUpdate':str(datetime.datetime.now().year)+str(datetime.datetime.now().month)+str(datetime.datetime.now().day),
        'status':'complete',
        'strategyId':strategy_name
    }})
    db.strategyBackTest.update_one({'strategy_name':strategy_name}, {'$set':{
        'lastUpdate':str(datetime.datetime.now().year)+str(datetime.datetime.now().month)+str(datetime.datetime.now().day),
        'privacy':'public',
        'strategyId':strategy_name
    }})

C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


从tradesUsed cache中踢出 20201208
从tradesUsed cache中踢出 20201207
从tradesUsed cache中踢出 20201204
从tradesUsed cache中踢出 20201203
TagBreak3MHighOpen 最后完整日期 20201202 新增记录 127 总记录 25108
['净利润增速连续三年大于20%', '季度净利润连续三年大于1亿', '跳空高开', '突破1年新高(开盘)', '突破3个月新高(开盘)', '均线多头排列', 'MACD的DIFF高于DEA', 'MACD的DIFF大于0', '交易量放大2倍', '流通市值大于100亿', '流通市值大于250亿', '流通市值大于500亿', '流通市值小于100亿', '流通市值小于50亿', '流通市值小于25亿', 'KDJ金叉', '十大股东持股比例小于20%', '十大股东持股比例20%~50%', '十大股东持股比例50%~80%', '十大股东持股比例大于80%', '交通运输', '休闲服务', '传媒', '公用事业', '农林牧渔', '化工', '医药生物', '商业贸易', '国防军工', '家用电器', '建筑材料', '建筑装饰', '房地产', '有色金属', '机械设备', '汽车', '电子', '电气设备', '纺织服装', '综合', '计算机', '轻工制造', '通信', '采掘', '钢铁', '银行', '非银金融', '食品饮料', '5日乖离率超买Ⅰ', '5日乖离率超买Ⅱ', '5日乖离率超卖Ⅰ', '5日乖离率正常', '3个月内涨幅>50%', '6个月内涨幅>50%', '1年内涨幅>50%', '5日乖离率超卖Ⅱ', '价格突破40周周线', '价格位于40周周线的0%-1%区间', '40周周线趋势向上', '不要SP500涨幅>2%', '不要SP500跌幅>2%']
盈利前10%交易 净利润增速连续三年大于20% 1.85
盈利前10%交易 季度净利润连续三年大于1亿 100.0
盈利前10%交易 跳空高开 37.04
盈利前10%交易 突破1年新高(开盘) 27.78
盈利前10%交易 突破3个月新高(开盘) 59.72
盈利前10%交易 均线多头排列 83.33

第一天跌停 季度净利润连续三年大于1亿 100.0
第一天跌停 跳空高开 0.0
第一天跌停 突破1年新高(开盘) 9.09
第一天跌停 突破3个月新高(开盘) 9.09
第一天跌停 均线多头排列 45.45
第一天跌停 MACD的DIFF高于DEA 0.0
第一天跌停 MACD的DIFF大于0 72.73
第一天跌停 交易量放大2倍 0.0
第一天跌停 流通市值大于100亿 90.91
第一天跌停 流通市值大于250亿 36.36
第一天跌停 流通市值大于500亿 18.18
第一天跌停 流通市值小于100亿 9.09
第一天跌停 流通市值小于50亿 0.0
第一天跌停 流通市值小于25亿 0.0
第一天跌停 KDJ金叉 0.0
第一天跌停 十大股东持股比例小于20% 9.09
第一天跌停 十大股东持股比例20%~50% 18.18
第一天跌停 十大股东持股比例50%~80% 63.64
第一天跌停 十大股东持股比例大于80% 9.09
第一天跌停 交通运输 18.18
第一天跌停 休闲服务 0.0
第一天跌停 传媒 0.0
第一天跌停 公用事业 0.0
第一天跌停 农林牧渔 9.09
第一天跌停 化工 9.09
第一天跌停 医药生物 9.09
第一天跌停 商业贸易 0.0
第一天跌停 国防军工 0.0
第一天跌停 家用电器 9.09
第一天跌停 建筑材料 9.09
第一天跌停 建筑装饰 0.0
第一天跌停 房地产 0.0
第一天跌停 有色金属 0.0
第一天跌停 机械设备 0.0
第一天跌停 汽车 9.09
第一天跌停 电子 9.09
第一天跌停 电气设备 0.0
第一天跌停 纺织服装 0.0
第一天跌停 综合 0.0
第一天跌停 计算机 9.09
第一天跌停 轻工制造 0.0
第一天跌停 通信 0.0
第一天跌停 采掘 0.0
第一天跌停 钢铁 0.0
第一天跌停 银行 0.0
第一天跌停 非银金融 0.0
第一天跌停 食品饮料 9.09
第一天跌停 5日乖离率超买Ⅰ 18.18
第一天跌停 5日乖离率超买Ⅱ 18.18
第一天跌停 5日乖离率超卖Ⅰ 0.0
第一天跌停 5日乖离率正常 81.82
第一天跌停 3个月内涨幅>50% 27.27
第一天跌停 6个月内涨幅>50% 54.55
第一天跌停 1年内涨幅>50% 72.73
第一天跌停 5日乖离率超卖Ⅱ

C:\ProgramData\Anaconda3\envs\research\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:231: RuntimeWarning: invalid value encountered in double_scalars
  

第1天10:35,第3天13:22,TagBreak3MHighOpen,tagNetProfit3YBiggerThan100M getPnl() at  0 / 2868
240
第1天10:35,第3天13:22,TagBreak3MHighOpen,tagNetProfit3YBiggerThan100M getPnl() at  0 / 2868


In [ ]:
#importlib.reload(rschLib)
#%load_ext line_profiler
#%lprun -f rschLib.getPnl rschLib.getPnl(dtes,tkrs, name, tradesUsed, inTime, otTime, dayOff, timeAsFloat)
